# Bike Sharing Assignment

#### Importing all important libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings
warnings.filterwarnings('ignore')

#### Importing the data

In [2]:
df = pd.read_csv("day.csv")
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


## Step 1: Reading, Understanding, and Visualising the Data

Taking care of null/missing values

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    int64  
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.4+ KB


So, no data to drop or impute

Adding a derived column of day in the date, and dropping the dteday columns.

In [4]:
df[ 'dteday' ] = pd.to_datetime( df[ 'dteday' ], format="%d-%m-%Y" )
df[ 'day' ] = df[ 'dteday' ].dt.strftime('%d')
df.drop(columns=['dteday'], inplace=True)
df.head()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,day
0,1,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,01
1,2,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,02
2,3,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349,03
3,4,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562,04
4,5,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600,05


Mapping Categorical Values as per the Description (from Readme.txt)

In [5]:
df['season'] = df['season'].map({
    1: 'spring',
    2: 'summer',
    3: 'fall',
    4: 'winter'
})
df['weekday'] = df['weekday'].map({
    0: 'Tues',
    1: 'Wed', 
    2: 'Thurs', 
    3: 'Fri', 
    4: 'Sat', 
    5: 'Sun', 
    6: 'Mon'
})
df['weathersit'] = df['weathersit'].map({
    1: 'best_weather',
    2: 'avg_weather',
    3: 'bad_weather',
    4: 'worst_weather'
})

df.head()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,day
0,1,spring,0,1,0,Mon,0,avg_weather,14.110847,18.18125,80.5833,10.749882,331,654,985,01
1,2,spring,0,1,0,Tues,0,avg_weather,14.902598,17.68695,69.6087,16.652113,131,670,801,02
2,3,spring,0,1,0,Wed,1,best_weather,8.050924,9.47025,43.7273,16.636703,120,1229,1349,03
3,4,spring,0,1,0,Thurs,1,best_weather,8.200000,10.60610,59.0435,10.739832,108,1454,1562,04
4,5,spring,0,1,0,Fri,1,best_weather,9.305237,11.46350,43.6957,12.522300,82,1518,1600,05


Checking the number of unique values in each column to figure out categorical and numeric columns

In [6]:
df.nunique()

instant       730
season          4
yr              2
mnth           12
holiday         2
weekday         7
workingday      2
weathersit      3
temp          498
atemp         689
hum           594
windspeed     649
casual        605
registered    678
cnt           695
day            31
dtype: int64

So,
- X numeric columns: temp, atemp, hum, windspeed
- X categorical columns: season, yr, mnth, holiday, weekday, workingday, weathersit
- X date columns: dteday
- Y(target) variable: cnt
- Y non useful variable: casual, registered
- index columns: instant

#### Separating the Categorical and Numercal X and y Columns

In [7]:
X_num_cols = ['temp', 'atemp', 'hum', 'windspeed']
X_cat_cols = ['season', 'yr', 'day', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']
other_cols = [ 'instant', 'casual', 'registered' ]

y_col = 'cnt'

Dropping other columns other than X and y

In [8]:
df.drop(columns=other_cols, inplace=True)

#### Analysis on X Numeric Columns

In [9]:
df[ X_num_cols ].describe()

,temp,atemp,hum,windspeed
count,730.000000,730.000000,730.000000,730.000000
mean,20.319259,23.726322,62.765175,12.763620
std,7.506729,8.150308,14.237589,5.195841
min,2.424346,3.953480,0.000000,1.500244
25%,13.811885,16.889713,52.000000,9.041650
50%,20.465826,24.368225,62.625000,12.125325
75%,26.880615,30.445775,72.989575,15.625589
max,35.328347,42.044800,97.250000,34.000021


In [ ]:
sns.pairplot(df[ X_num_cols + [y_col] ])
plt.show()

2 major takeaways:
- temp and atemp seems to be highly correlated.
- There seems to be a small linear relationship bw target variable cnt, and the variables temp and atemp.

In [ ]:
sns.heatmap( df[ X_num_cols + [y_col] ].corr(), annot=True )
plt.show()

The above mentioned 2 points are confirmed. In addition to that, 
- There also seems to be a negative correlation of target variable with the windspeed
- There seems to be no linear correlation between humidity and target variable cnt

We won't be acting on these inferences at the moment. We will do so during model training and feature selection phase

#### Analysis on X Categorical Columns

In [ ]:
for col in X_cat_cols:
    sns.boxplot(data=df, x=col, y='cnt')
    plt.show()

Following inferences can be made:
- business seems to be worse during the spring and best during the fall. It increases from spring to fall, and decreases from fall to spring.
- More business happenned in 2019 than 2018
- no observable trend between the days and business
- Relationship with months and business mimicks the relation seen with seasons.
- Business seems to low during holidays
- no observable trend bw weekdays and business
- business seems to do a bit better during workdays
- business does better with better weather

# Step 2: Preparing The Data for Modelling

## Encoding

In [ ]:
for col in X_cat_cols:
    print(f"Col: {col} | Unique values: {df[ col ].unique()}")

So,

In [ ]:
X_cat_cols_wth_mult_cats = [ 'season', 'day', 'mnth', 'weekday', 'weathersit' ]

In [ ]:
for col in X_cat_cols_wth_mult_cats:
    df[ col ] = df[ col ].astype(str)

In [ ]:
df_dummy_vars = pd.get_dummies(df[ X_cat_cols_wth_mult_cats ], drop_first=True, dtype=int)
collist_dummy_vars = list(df_dummy_vars.columns)

df.drop(columns=X_cat_cols_wth_mult_cats, inplace=True)

df = pd.concat([ df, df_dummy_vars ], axis=1)

df.columns

Now the X categorical columns becomes

In [ ]:
X_cat_cols = [ 'yr', 'holiday', 'workingday' ] + collist_dummy_vars
X_cat_cols

## Splitting into Train and Test Set

In [ ]:
df_train, df_test = train_test_split(df, train_size=0.8, random_state=100)

print(df_train.shape)
print(df_test.shape)

In [ ]:
y_train = df_train.pop(y_col)
X_train = df_train
print(y_train.shape)
print(X_train.shape)

In [ ]:
y_test = df_test.pop(y_col)
X_test = df_test

print(y_test.shape)
print(X_test.shape)

## Re-scaling the Features

Using Min-Max scaling, so that it will take care of the outliers

In [ ]:
scaler = MinMaxScaler()
X_train[ X_num_cols ] = scaler.fit_transform( X_train[ X_num_cols ] )
X_train.describe()

# Step 3: Building the Model

#### First Draft of the Model

In [ ]:
X_train_sm = sm.add_constant(X_train)

lr = sm.OLS(y_train, X_train_sm)
lr_model = lr.fit()

lr_model.summary()

As is evident, lot of variables can be eliminated using feature selection.

First doing Coarse Tuning using Automated Approach

## Feature Selection

### Feature Selection: Coarse Tuning using RFE

Getting no of RFE high ranking columns vs R2 Score of the model created

In [ ]:
list_ncols_vs_r2 = []

for ncols in range(1, len(X_train.columns)+1):
    lr = LinearRegression()
    rfe = RFE(lr, n_features_to_select=ncols)
    rfe = rfe.fit(X_train, y_train)
    
    rfe_y_train_pred = rfe.predict(X_train)
    rfe_train_r2_score = r2_score(y_train, rfe_y_train_pred)
    
    list_ncols_vs_r2 += [ [ncols, rfe_train_r2_score] ]
    
df_ncols_vs_r2 = pd.DataFrame(list_ncols_vs_r2, columns=[ 'no_of_rfe_high_ranking_columns', 'r2_score' ])
df_ncols_vs_r2

In [ ]:
plt.figure(figsize=(20,5))

plt.plot(df_ncols_vs_r2['no_of_rfe_high_ranking_columns'], df_ncols_vs_r2['r2_score'], label='ncols_vs_r2score')
plt.axvline(x=13, color='r', linewidth=0.5, label='no_change_after_this_point')

plt.xlabel('no_of_rfe_high_ranking_columns')
plt.ylabel('r2_score')

plt.xticks(np.arange(min(df_ncols_vs_r2['no_of_rfe_high_ranking_columns']), \
                     max(df_ncols_vs_r2['no_of_rfe_high_ranking_columns'])+1, 1))
plt.legend()
plt.show()

As is evident from the dataframe and graph above, not much difference in R2 score after taking 13 RFE highest ranked variables

So, its logical to take 13 high ranked columns for further fine tuning

But before that, checking if we are missing any business important variables in the rest of the columns

In [ ]:
lr = LinearRegression()
rfe = RFE(lr, n_features_to_select=1)
rfe = rfe.fit(X_train, y_train)

list_rfe_col_ranking = list(zip( X_train.columns, rfe.ranking_ ))
df_rfe_col_ranking = pd.DataFrame(list_rfe_col_ranking, columns=['column', 'rfe_rank'])
df_rfe_col_ranking.sort_values(inplace=True, by='rfe_rank', ignore_index=True)
df_rfe_col_ranking

From all the EDA done previously, seasons did seem to have an impact in the target variable.

If we take only till the first 13 ranked columns, we will be dropping the season_spring column. <br>
From the EDA done before, it seemed that seasons did play an important part. <br>
So, taking till the first 21 ranked columns so as to also include season_spring.

In [ ]:
lr = LinearRegression()
rfe = RFE(lr, n_features_to_select=21)
rfe = rfe.fit(X_train, y_train)

X_rfe_filt_cols = list(X_train.columns[ rfe.support_ ])
X_rfe_filt_cols

### Feature Selection: Fine Tuning

We take RFE filtered columns as first draft of X columns after feature selection 

In [ ]:
X_train_sm = sm.add_constant( X_train[ X_rfe_filt_cols ] )
lr = sm.OLS(y_train, X_train_sm)
lr_model = lr.fit()

lr_model.summary()

In [ ]:
list_col_vs_vif = []
for idx, col in enumerate(X_rfe_filt_cols):
    vif = variance_inflation_factor(X_train[ X_rfe_filt_cols ].values, idx)
    list_col_vs_vif += [ [col, vif] ]
    
df_col_vs_vif = pd.DataFrame(list_col_vs_vif, columns=[ 'column', 'VIF' ])
df_col_vs_vif

atemp is insignificant, since it has a p-value greater than 0.05.

So, creating second draft of X columns that does not contain atemp

##### 2nd Column Draft

In [ ]:
X_rfe_filt_cols = [ col for col in X_rfe_filt_cols if col != 'atemp' ]
X_rfe_filt_cols

In [ ]:
X_train_sm = sm.add_constant( X_train[ X_rfe_filt_cols ] )
lr = sm.OLS(y_train, X_train_sm)
lr_model = lr.fit()

lr_model.summary()

In [ ]:
list_col_vs_vif = []
for idx, col in enumerate(X_rfe_filt_cols):
    vif = variance_inflation_factor(X_train[ X_rfe_filt_cols ].values, idx)
    list_col_vs_vif += [ [col, vif] ]
    
df_col_vs_vif = pd.DataFrame(list_col_vs_vif, columns=[ 'column', 'VIF' ])
df_col_vs_vif

All the p-values of variables are below 0.05.<br>
But there are correlations bw the columns in the second draft

workingday has the highest VIF, and is > 5.

So, creating a 3rd draft dropping workingday

##### 3rd Column Draft

In [ ]:
X_rfe_filt_cols = [ col for col in X_rfe_filt_cols if col != 'workingday' ]
X_rfe_filt_cols

In [ ]:
X_train_sm = sm.add_constant( X_train[ X_rfe_filt_cols ] )
lr = sm.OLS(y_train, X_train_sm)
lr_model = lr.fit()

lr_model.summary()

In [ ]:
list_col_vs_vif = []
for idx, col in enumerate(X_rfe_filt_cols):
    vif = variance_inflation_factor(X_train[ X_rfe_filt_cols ].values, idx)
    list_col_vs_vif += [ [col, vif] ]
    
df_col_vs_vif = pd.DataFrame(list_col_vs_vif, columns=[ 'column', 'VIF' ])
df_col_vs_vif

weekday_Mon has the highest p-value and is > 0.05

So, creating a 4th draft dropping weekday_Mon

##### 4rd Column Draft

In [ ]:
X_rfe_filt_cols = [ col for col in X_rfe_filt_cols if col != 'weekday_Mon' ]
X_rfe_filt_cols

In [ ]:
X_train_sm = sm.add_constant( X_train[ X_rfe_filt_cols ] )
lr = sm.OLS(y_train, X_train_sm)
lr_model = lr.fit()

lr_model.summary()

In [ ]:
list_col_vs_vif = []
for idx, col in enumerate(X_rfe_filt_cols):
    vif = variance_inflation_factor(X_train[ X_rfe_filt_cols ].values, idx)
    list_col_vs_vif += [ [col, vif] ]
    
df_col_vs_vif = pd.DataFrame(list_col_vs_vif, columns=[ 'column', 'VIF' ])
df_col_vs_vif

All p-values < 0.05 <br>
But there are correlations > 5!

hum has the highest VIF, and >5. So dropping hum

##### 5rd Column Draft

In [ ]:
X_rfe_filt_cols = [ col for col in X_rfe_filt_cols if col != 'hum' ]
X_rfe_filt_cols

In [ ]:
X_train_sm = sm.add_constant( X_train[ X_rfe_filt_cols ] )
lr = sm.OLS(y_train, X_train_sm)
lr_model = lr.fit()

lr_model.summary()

In [ ]:
list_col_vs_vif = []
for idx, col in enumerate(X_rfe_filt_cols):
    vif = variance_inflation_factor(X_train[ X_rfe_filt_cols ].values, idx)
    list_col_vs_vif += [ [col, vif] ]
    
df_col_vs_vif = pd.DataFrame(list_col_vs_vif, columns=[ 'column', 'VIF' ])
df_col_vs_vif

All p-values < 0.05 <br>
But there are correlations > 5!

temp has high correlation. But choosing to not drop it, since EDA had shown a good relation with the target variable

So, choosing to drop day_20 due to its near 0.05 p-value

##### 6rd Column Draft

In [ ]:
X_rfe_filt_cols = [ col for col in X_rfe_filt_cols if col != 'day_20' ]
X_rfe_filt_cols

In [ ]:
X_train_sm = sm.add_constant( X_train[ X_rfe_filt_cols ] )
lr = sm.OLS(y_train, X_train_sm)
lr_model = lr.fit()

lr_model.summary()

In [ ]:
list_col_vs_vif = []
for idx, col in enumerate(X_rfe_filt_cols):
    vif = variance_inflation_factor(X_train[ X_rfe_filt_cols ].values, idx)
    list_col_vs_vif += [ [col, vif] ]
    
df_col_vs_vif = pd.DataFrame(list_col_vs_vif, columns=[ 'column', 'VIF' ])
df_col_vs_vif

temp variable have > 5 VIF. But not choosing to drop (reason stated in previous draft)

But choosing to drop the next biggest VIF variable windspeed, <br>
since it does have high correlation with temp (from EDA)

##### 7rd Column Draft

In [ ]:
X_rfe_filt_cols = [ col for col in X_rfe_filt_cols if col != 'windspeed' ]
X_rfe_filt_cols

In [ ]:
X_train_sm = sm.add_constant( X_train[ X_rfe_filt_cols ] )
lr = sm.OLS(y_train, X_train_sm)
lr_model = lr.fit()

lr_model.summary()

In [ ]:
list_col_vs_vif = []
for idx, col in enumerate(X_rfe_filt_cols):
    vif = variance_inflation_factor(X_train[ X_rfe_filt_cols ].values, idx)
    list_col_vs_vif += [ [col, vif] ]
    
df_col_vs_vif = pd.DataFrame(list_col_vs_vif, columns=[ 'column', 'VIF' ])
df_col_vs_vif

##### Finalized Column Draft

In [ ]:
X_final_draft_cols = X_rfe_filt_cols

lr_model_final = lr_model

lr_model_final.summary()

So, the equation looks like<br>
y = 526.45<br>+2001.47\*yr<br>-1045.42\*holiday<br>+4180.74\*temp<br>-508.86\*season_spring<br>+603.29\*season_summer<br>+833.91\*season_winter<br>+387.93\*day_11<br>+452.76\*day_16<br>+388.27\*day_17<br>+433.68\*mnth_10<br>+340.26\*mnth_8<br>+912.70\*mnth_9<br>-460.89\*weekday_Tues<br>-2076.14\*weathersit_bad_weather<br>+707.02\*weathersit_best_weather

# Step 4: Residual Analysis

In [ ]:
X_train_final_draft_sm = sm.add_constant(X_train[ X_final_draft_cols ])
y_train_pred = lr_model_final.predict( X_train_final_draft_sm )

res = y_train - y_train_pred

In [ ]:
sns.distplot(res)
plt.show()

As is evident from the graph,
- The residuals are centered around 0
- The residuals follow a normal distribution

So, our assumptions are valid

# Step 5: Predictions and Evaluations on the Test Set

## Rescaling

In [ ]:
X_test[ X_num_cols ] = scaler.transform(X_test[ X_num_cols ])
X_test[ X_num_cols ].describe()

## Predict

In [ ]:
X_test_final_draft_sm = sm.add_constant( X_test[ X_final_draft_cols ] )
y_test_pred = lr_model_final.predict( X_test_final_draft_sm )

## Test R2-Score

In [ ]:
test_r2_score = r2_score(y_true=y_test, y_pred=y_test_pred)
test_r2_score